In [1]:
import polars as pl
import altair as alt

In [236]:
import warnings
from src.kristi_promin import kristi_promin
from src.me_to_neurazi import me_to_neurazi
from src.alt_friendly import alt_friendly

pl.Config.set_tbl_rows(150)
pl.Config.set_fmt_str_lengths(150)
pl.Config.set_tbl_width_chars(200)

alt.themes.register('irozhlas', kristi_promin)
alt.themes.enable('irozhlas')
warnings.filterwarnings('ignore')

In [3]:
df = pl.scan_parquet(
    "data/projevy.parquet"
).filter(
    pl.col('datum').dt.year() >= 1918
).filter(
    ~(pl.col("komora_komplet").str.contains("Senát") 
      | 
      pl.col('komora_komplet').str.contains('Společn') 
      | 
      pl.col('komora').str.contains('psse')
     )
).drop_nulls(
    subset=['datum']
).with_columns(
    pl.col("text").str.count_matches(" ").alias("pocet_slov")
).sort(
    by='datum'
).join(
    pl.scan_parquet(
        "data/projevy_jazyky.parquet",
    ),
    how="left",
    on="text"
).filter(
    pl.col("jazyk") != "sk"
).collect()

In [4]:
len(df)

699208

In [5]:
def vyvoj_vyskytu(hledani, popisek):
    
    return df.with_columns(
        pl.col("text").str.count_matches(hledani).alias("vyskytu")
    ).group_by_dynamic(
        index_column='datum',every='1y'
    ).agg(
        pl.col("vyskytu").sum()
    ).join(
    df.group_by_dynamic(
        index_column='datum',every='1y'
    ).agg(
        pl.col("pocet_slov").sum()
    ),
    how="left",
    on="datum"
).with_columns(
    (pl.col("vyskytu") / pl.col("pocet_slov")).alias("pomer")
).with_columns(
    pl.lit(popisek).alias("co")
)

In [6]:
def graf(frejm):
    return alt.Chart(
    frejm.to_pandas(),
    width=800,
    height=100
).mark_bar(
).encode(
    alt.X("datum:T"),
    alt.Y("pomer:Q"),
    alt.Row("co:N")
)
    

In [7]:
graf(vyvoj_vyskytu("(?i)v televizi","v televizi"))

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [8]:
graf(vyvoj_vyskytu("(?i)v r[aá]diu","v rádiu"))

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [9]:
graf(vyvoj_vyskytu("(?i)v rozhlase","v rozhlase"))

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [10]:
graf(vyvoj_vyskytu("(?i)v novinách","v novinách"))

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [11]:
graf(vyvoj_vyskytu("(?i)v knížce","v knížce"))

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [12]:
graf(vyvoj_vyskytu("(?i)ve{0,1} \w{0,20}\s{0,1}románu","v románu"))

<>:1: SyntaxWarning: invalid escape sequence '\w'
<>:1: SyntaxWarning: invalid escape sequence '\w'
C:\Users\micha\AppData\Local\Temp\ipykernel_37364\725468792.py:1: SyntaxWarning: invalid escape sequence '\w'
  graf(vyvoj_vyskytu("(?i)ve{0,1} \w{0,20}\s{0,1}románu","v románu"))
C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [13]:
def hledej(text, n=3, delka=200, frejm=df):
    nalezy = frejm.sort(by="datum").filter(pl.col("text").str.contains(text)).head(n).iter_rows()
    for cislo, radek in enumerate(nalezy):
        print(f'Kdo: {radek[0]}')
        print(f'Kde: {radek[1]}')
        print(f'Kdy: {radek[9]}')
        
        # Split the text around the search term
        parts = re.split(text, radek[3])
        
        # Get the part before the search term (safe if empty)
        before = parts[0] if len(parts) > 0 else ""
        # Get the part after the search term (safe if empty)
        after = parts[1] if len(parts) > 1 else ""
        
        # Take up to 200 characters from before and after, but don't break if shorter
        before_snippet = before[-delka:] if len(before) > delka else before
        after_snippet = after[:delka] if len(after) > delka else after
        nalez = re.search(text, radek[3]).group()
        
        uryvek = before_snippet + nalez + after_snippet
        print(f'Co: {uryvek}')
        
        if cislo != 2:  # Changed from 3 to 2 since enumerate starts at 0
            print("***")

In [14]:
import re

In [15]:
hledej("v románu")

Kdo: Poslanec Miloslav Ransdorf
Kde: 1998ps_stenprot_013schuz_s013456.htm
Kdy: 1999-06-08
Co: tní péče je příkladem, který příliš vhodný není. Někdy se mi zdá, že přístup evropských byrokratů je podobný tomu, jak postupoval výbor na oslavu 70letého mírového panování císaře Františka Josefa I. v románu Muž bez vlastností Roberta Musila. Vypukla první světová válka, císař zemřel, ale výbor pro oslavu jeho 70letého mírového panování pracoval dál. I když se změnily některé podmínky, přesto určité norm
***
Kdo: Poslanec Vladimír Zlínský
Kde: 2021ps_stenprot_074schuz_s074087.htm
Kdy: 2023-09-06
Co: r informací prostřednictvím digitalizace společnosti, analýza těchto dat, systémy umělé inteligence, efektivní centrální řízení společnosti v míře, která byla dříve těžko představitelná a známá spíše v románu George Orwella 1984. Záměrně tento román tady zmiňuji, já jsem ho nedávno četl - přiznám se, že jsem velký čtenář, ale měl jsem deficit, že jsem ho nečetl, protože jsem byl před tím varován, 

In [16]:
graf(vyvoj_vyskytu("(?i)v knize","v knize"))

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [17]:
graf(vyvoj_vyskytu("(?i)v časopis","v časopisu"))

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [18]:
graf(vyvoj_vyskytu("(?i)","v časopisu"))

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [19]:
graf(vyvoj_vyskytu("(?i)na internetu","na internetu"))

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [20]:
graf(vyvoj_vyskytu("(?i)whatsapp","whatsapp"))

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [21]:
import re

In [22]:
def jak_napsal(retezec):
    return re.findall(r"(?i)jak (napsala{0,1}|píše) \w{1,20} \w{1,20} \w{1,20}",retezec)

In [23]:
jak_napsal("protože jak napsala František Novák v knize")

['napsala']

In [24]:
def v_knize(retezec):
    return re.findall(r"jak (se píše|stojí) v knize \w{1,20} \w{1,20} \w{1,20}",retezec)

In [25]:
len(df) / 1000

699.208

In [26]:
%%timeit
df.sample(729).with_columns(pl.col('text').map_elements(jak_napsal, return_dtype=pl.List(pl.String)).alias('jak_napsal'))

77.7 ms ± 2.79 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [27]:
df = df.with_columns(pl.col('text').map_elements(jak_napsal, return_dtype=pl.List(pl.String)).alias('jak_napsal'))

In [28]:
df.explode('jak_napsal').group_by('jak_napsal').len().sort(by='len',descending=True)

jak_napsal,len
str,u32
null,699162
"""píše""",33
"""napsal""",13
"""napsala""",2


In [29]:
df = df.with_columns(pl.col('text').map_elements(v_knize, return_dtype=pl.List(pl.String)).alias('v_knize'))

In [30]:
def spisovatel(retezec):
    return [' '.join(x.split(' ')[1:]) for x in re.findall(r"(?i)spisovatel\w{0,5} \w{1,15} \w{1,15}\.{0,1} \w{1,15}",retezec)]

In [31]:
spisovatel('Slovy spisovatele Marka P. Twaina: srát na slávu jak na placatý kámen.')

['Marka P. Twaina']

In [32]:
df = df.with_columns(pl.col('text').map_elements(spisovatel, return_dtype=pl.List(pl.String)).alias('spisovatelstvo'))

In [33]:
df.group_by('spisovatelstvo').len().sort(by='len',descending=True)

spisovatelstvo,len
list[str],u32
[],698907
"[""při Svatoboru v""]",3
"[""rozdělovala podle klíče""]",3
"[""a finančního komisaře""]",3
"[""jsou poukázáni na""]",2
"[""Karel Čapek napsal""]",2
"[""a výtvarných umělců""]",2
"[""Novák o nich""]",1
"[""všeho druhu jim""]",1


In [34]:
graf(vyvoj_vyskytu("Kar\w{1,5} Čap\w{1,10}","Karel Čapek"))

<>:1: SyntaxWarning: invalid escape sequence '\w'
<>:1: SyntaxWarning: invalid escape sequence '\w'
C:\Users\micha\AppData\Local\Temp\ipykernel_37364\2086449438.py:1: SyntaxWarning: invalid escape sequence '\w'
  graf(vyvoj_vyskytu("Kar\w{1,5} Čap\w{1,10}","Karel Čapek"))
C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [35]:
graf(vyvoj_vyskytu("Milan\w{0,5} Kunder\w{1,10}","Milan Kundera"))

<>:1: SyntaxWarning: invalid escape sequence '\w'
<>:1: SyntaxWarning: invalid escape sequence '\w'
C:\Users\micha\AppData\Local\Temp\ipykernel_37364\507957919.py:1: SyntaxWarning: invalid escape sequence '\w'
  graf(vyvoj_vyskytu("Milan\w{0,5} Kunder\w{1,10}","Milan Kundera"))
C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [36]:
graf(vyvoj_vyskytu("Franz\w{0,5} Kafk\w{1,10}","Franz Kafka"))

<>:1: SyntaxWarning: invalid escape sequence '\w'
<>:1: SyntaxWarning: invalid escape sequence '\w'
C:\Users\micha\AppData\Local\Temp\ipykernel_37364\1969676538.py:1: SyntaxWarning: invalid escape sequence '\w'
  graf(vyvoj_vyskytu("Franz\w{0,5} Kafk\w{1,10}","Franz Kafka"))
C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [37]:
graf(vyvoj_vyskytu("Jaros\w{1,10} Haš[ek]{1,10}","Jaroslav Hašek"))

<>:1: SyntaxWarning: invalid escape sequence '\w'
<>:1: SyntaxWarning: invalid escape sequence '\w'
C:\Users\micha\AppData\Local\Temp\ipykernel_37364\4101695588.py:1: SyntaxWarning: invalid escape sequence '\w'
  graf(vyvoj_vyskytu("Jaros\w{1,10} Haš[ek]{1,10}","Jaroslav Hašek"))
C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [38]:
graf(vyvoj_vyskytu("Božen\w{1,10} Němcov{1,10}","Božena Němcová"))

<>:1: SyntaxWarning: invalid escape sequence '\w'
<>:1: SyntaxWarning: invalid escape sequence '\w'
C:\Users\micha\AppData\Local\Temp\ipykernel_37364\2089479426.py:1: SyntaxWarning: invalid escape sequence '\w'
  graf(vyvoj_vyskytu("Božen\w{1,10} Němcov{1,10}","Božena Němcová"))
C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [39]:
graf(vyvoj_vyskytu("Jaros\w{1,10} Seifert{1,10}","Jaroslav Seifert"))

<>:1: SyntaxWarning: invalid escape sequence '\w'
<>:1: SyntaxWarning: invalid escape sequence '\w'
C:\Users\micha\AppData\Local\Temp\ipykernel_37364\2866260283.py:1: SyntaxWarning: invalid escape sequence '\w'
  graf(vyvoj_vyskytu("Jaros\w{1,10} Seifert{1,10}","Jaroslav Seifert"))
C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [40]:
graf(vyvoj_vyskytu("Juli\w{1,10} Fučí","Julius Fučík"))

<>:1: SyntaxWarning: invalid escape sequence '\w'
<>:1: SyntaxWarning: invalid escape sequence '\w'
C:\Users\micha\AppData\Local\Temp\ipykernel_37364\2295232114.py:1: SyntaxWarning: invalid escape sequence '\w'
  graf(vyvoj_vyskytu("Juli\w{1,10} Fučí","Julius Fučík"))
C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [41]:
graf(vyvoj_vyskytu("Václav\w{0,10} Hav[el]\w{1,10}","Václav Havel"))

<>:1: SyntaxWarning: invalid escape sequence '\w'
<>:1: SyntaxWarning: invalid escape sequence '\w'
C:\Users\micha\AppData\Local\Temp\ipykernel_37364\1588657869.py:1: SyntaxWarning: invalid escape sequence '\w'
  graf(vyvoj_vyskytu("Václav\w{0,10} Hav[el]\w{1,10}","Václav Havel"))
C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [42]:
graf(vyvoj_vyskytu("Jan\w{0,10} Nerud",""))

<>:1: SyntaxWarning: invalid escape sequence '\w'
<>:1: SyntaxWarning: invalid escape sequence '\w'
C:\Users\micha\AppData\Local\Temp\ipykernel_37364\2886232089.py:1: SyntaxWarning: invalid escape sequence '\w'
  graf(vyvoj_vyskytu("Jan\w{0,10} Nerud",""))
C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [43]:
graf(vyvoj_vyskytu("Kar\w{1,10} Hyn\w{1,10} Mách\w{1,10}",""))

<>:1: SyntaxWarning: invalid escape sequence '\w'
<>:1: SyntaxWarning: invalid escape sequence '\w'
C:\Users\micha\AppData\Local\Temp\ipykernel_37364\2934549395.py:1: SyntaxWarning: invalid escape sequence '\w'
  graf(vyvoj_vyskytu("Kar\w{1,10} Hyn\w{1,10} Mách\w{1,10}",""))
C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [44]:
graf(vyvoj_vyskytu("Goethe",""))

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [45]:
graf(vyvoj_vyskytu("Bibl\w{1,10}",""))

<>:1: SyntaxWarning: invalid escape sequence '\w'
<>:1: SyntaxWarning: invalid escape sequence '\w'
C:\Users\micha\AppData\Local\Temp\ipykernel_37364\3902608589.py:1: SyntaxWarning: invalid escape sequence '\w'
  graf(vyvoj_vyskytu("Bibl\w{1,10}",""))
C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [46]:
graf(vyvoj_vyskytu("[oO]rwell",""))

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [47]:
graf(vyvoj_vyskytu("Kar\w{1,10} Havl\w{1,10} Borovsk\w{1,10}",""))

<>:1: SyntaxWarning: invalid escape sequence '\w'
<>:1: SyntaxWarning: invalid escape sequence '\w'
C:\Users\micha\AppData\Local\Temp\ipykernel_37364\960932723.py:1: SyntaxWarning: invalid escape sequence '\w'
  graf(vyvoj_vyskytu("Kar\w{1,10} Havl\w{1,10} Borovsk\w{1,10}",""))
C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [48]:
graf(vyvoj_vyskytu("Aldou\w{1,10} Huxl\w{1,10}",""))

<>:1: SyntaxWarning: invalid escape sequence '\w'
<>:1: SyntaxWarning: invalid escape sequence '\w'
C:\Users\micha\AppData\Local\Temp\ipykernel_37364\1871919798.py:1: SyntaxWarning: invalid escape sequence '\w'
  graf(vyvoj_vyskytu("Aldou\w{1,10} Huxl\w{1,10}",""))
C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [49]:
graf(vyvoj_vyskytu("Masaryk\w{0,10}",""))

<>:1: SyntaxWarning: invalid escape sequence '\w'
<>:1: SyntaxWarning: invalid escape sequence '\w'
C:\Users\micha\AppData\Local\Temp\ipykernel_37364\2111310961.py:1: SyntaxWarning: invalid escape sequence '\w'
  graf(vyvoj_vyskytu("Masaryk\w{0,10}",""))
C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [50]:
graf(vyvoj_vyskytu("Arendt\w{0,10}",""))

<>:1: SyntaxWarning: invalid escape sequence '\w'
<>:1: SyntaxWarning: invalid escape sequence '\w'
C:\Users\micha\AppData\Local\Temp\ipykernel_37364\1530276943.py:1: SyntaxWarning: invalid escape sequence '\w'
  graf(vyvoj_vyskytu("Arendt\w{0,10}",""))
C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [51]:
graf(vyvoj_vyskytu("(?i)superman\w{0,10}",""))

<>:1: SyntaxWarning: invalid escape sequence '\w'
<>:1: SyntaxWarning: invalid escape sequence '\w'
C:\Users\micha\AppData\Local\Temp\ipykernel_37364\568270949.py:1: SyntaxWarning: invalid escape sequence '\w'
  graf(vyvoj_vyskytu("(?i)superman\w{0,10}",""))
C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [52]:
graf(vyvoj_vyskytu("(?i)batman\w{0,10}",""))

<>:1: SyntaxWarning: invalid escape sequence '\w'
<>:1: SyntaxWarning: invalid escape sequence '\w'
C:\Users\micha\AppData\Local\Temp\ipykernel_37364\2279198183.py:1: SyntaxWarning: invalid escape sequence '\w'
  graf(vyvoj_vyskytu("(?i)batman\w{0,10}",""))
C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [53]:
graf(vyvoj_vyskytu("(?i)disney\w{0,10}",""))

<>:1: SyntaxWarning: invalid escape sequence '\w'
<>:1: SyntaxWarning: invalid escape sequence '\w'
C:\Users\micha\AppData\Local\Temp\ipykernel_37364\3759340266.py:1: SyntaxWarning: invalid escape sequence '\w'
  graf(vyvoj_vyskytu("(?i)disney\w{0,10}",""))
C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [54]:
graf(vyvoj_vyskytu("(?i)Keynes\w{0,10}",""))

<>:1: SyntaxWarning: invalid escape sequence '\w'
<>:1: SyntaxWarning: invalid escape sequence '\w'
C:\Users\micha\AppData\Local\Temp\ipykernel_37364\233220506.py:1: SyntaxWarning: invalid escape sequence '\w'
  graf(vyvoj_vyskytu("(?i)Keynes\w{0,10}",""))
C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [55]:
graf(vyvoj_vyskytu("(?i)orwell\w{0,10}",""))

<>:1: SyntaxWarning: invalid escape sequence '\w'
<>:1: SyntaxWarning: invalid escape sequence '\w'
C:\Users\micha\AppData\Local\Temp\ipykernel_37364\2204767158.py:1: SyntaxWarning: invalid escape sequence '\w'
  graf(vyvoj_vyskytu("(?i)orwell\w{0,10}",""))
C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [56]:
df.filter(pl.col("text").str.contains("(?i)orwell")).group_by("mluvci").len().sort(by="len",descending=True)

mluvci,len
str,u32
"""Poslanec Jiří Kobza""",7
"""Poslanec Tomio Okamura""",5
"""Poslanec Radek Vondráček""",5
"""Poslanec Walter Bartoš""",4
"""Poslanec Zdeněk Jičínský""",4
"""Poslanec David Rath""",4
"""Místopředseda PSP Ivan Langer""",4
"""Poslanec Jan Klán""",3
"""Poslanec Marek Benda""",3


In [57]:
hledej("(?i)orwell")

Kdo: Poslanec Jiří Šoler
Kde: 1993ps_stenprot_008schuz_s008033.htm
Kdy: 1993-04-22
Co: í rudých suken i primitivní politické agitky v našich obcích i z občanského života, i když jejich nahrazení komerční reklamou je asi stejně odstrašující, ale alespoň odpovídá spíš duchu Warholovu než Orwellovu a přineslo tedy určitou změnu odpovídající duchu národa. Bohužel kromě demokratického smýšlení zůstali u nás ještě konzervativní jedinci, kteří nepochopili duch demokracie a symbol KSČ pouze nahra
***
Kdo: Poslanec Jiří Šoler
Kde: 1993ps_stenprot_023schuz_s023020.htm
Kdy: 1994-09-29
Co: ít v demokratické a svobodné společnosti. Bohužel převrat v listopadu 1989 nepřinesl kýženou změnu, ale pouze předem domluvenou výměnu "těch nahoře" za "ty uprostřed" ve smyslu absurdní novely George Orwella nazvané "1984" a hlavní "demokratickou" starostí staronových mocných bylo zajistit, aby se k moci nedostaly nežádoucí nezávislé a skutečně demokratické síly. A to často za téměř jakoukoli cenu. Pozn
***
Kdo:

In [58]:
graf(vyvoj_vyskytu("(?i)Friedman\w{0,10}",""))

<>:1: SyntaxWarning: invalid escape sequence '\w'
<>:1: SyntaxWarning: invalid escape sequence '\w'
C:\Users\micha\AppData\Local\Temp\ipykernel_37364\1886130067.py:1: SyntaxWarning: invalid escape sequence '\w'
  graf(vyvoj_vyskytu("(?i)Friedman\w{0,10}",""))
C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [59]:
graf_ekonomu = alt.Chart(
    pl.concat(
        [
            vyvoj_vyskytu("Keynes","Keynes"),
            vyvoj_vyskytu("Friedman","Friedman"),
            vyvoj_vyskytu("Marx","Marx")
        ]
    ).to_pandas(),
    width=500,
    height=100
).mark_bar(
).encode(
    alt.X("datum:T"),
    alt.Y("pomer:Q"),
    alt.Row("co:N", sort=['Marx','Keynes','Friedman'])
)

graf_ekonomu

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

## Syntéza médií

In [226]:
media_pokupe = pl.concat(
        [
            vyvoj_vyskytu(r"(?i)[^\w]v televiz","v televizi"),
            vyvoj_vyskytu(r"(?i)[^\w](v rozhlas|v r[áa]diu)","v rozhlase / rádiu"),
            vyvoj_vyskytu(r"(?i)[^\w]v časopis","v časopise"),
            vyvoj_vyskytu(r"(?i)[^\w]v novinách","v novinách"),
            vyvoj_vyskytu(r"(?i)[^\w]ve{0,1} \w{0,20}\s{0,1}knize","v knize"),
            vyvoj_vyskytu(r"(?i)[^\w]ve{0,1} \w{0,20}\s{0,1}románu","v románu"),
            vyvoj_vyskytu(r"(?i)[^\w]na internetu","na internetu"),
            vyvoj_vyskytu(r"(?i)[^\w]na facebooku","na Facebooku"),
            vyvoj_vyskytu(r"(?i)[^\w]youtu","YouTube"),
            vyvoj_vyskytu(r"(?i)tweet","Twitter"),
            vyvoj_vyskytu(r"DVTV","DVTV"),
        ]
)

In [62]:
graf(media_pokupe)

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [63]:
media_pokupe

datum,vyskytu,pocet_slov,pomer,co
date,u32,u32,f64,str
1918-01-01,0,172010,0.0,"""v televizi"""
1919-01-01,0,1547459,0.0,"""v televizi"""
1920-01-01,0,1557497,0.0,"""v televizi"""
1921-01-01,0,1814119,0.0,"""v televizi"""
1922-01-01,0,1739976,0.0,"""v televizi"""
1923-01-01,0,1131296,0.0,"""v televizi"""
1924-01-01,0,1386775,0.0,"""v televizi"""
1925-01-01,0,989387,0.0,"""v televizi"""
1926-01-01,0,1629693,0.0,"""v televizi"""


In [227]:
media_pokupe_fin = media_pokupe.join(
    media_pokupe.group_by(
        "datum"
    ).agg(
        pl.col("pomer").max()
    ).with_columns(
        pl.lit(True).alias("max")
    ), 
    how="left",
    on=["datum","pomer"]
).with_columns(
    pl.when(pl.col('max').is_null()).then(pl.lit(False)).otherwise(pl.lit(True))
)



In [166]:
print(media_pokupe.select(pl.col('co')).unique().to_series().to_list())

['v rádiu', 'Twitter', 'na Facebooku', 'v knize', 'v televizi', 'YouTube', 'v novinách', 'DVTV', 'v románu', 'v rozhlase', 'na internetu', 'v časopise']


In [230]:
poradi_medii = ['v novinách','v knize','v časopise','v televizi','v rozhlase / rádiu','na internetu']

In [251]:
graf_medii = alt.Chart(
    media_pokupe_fin.filter(pl.col('co').is_in(poradi_medii)).to_pandas(),
    width=300,
    height=60,
    title=alt.Title(
        "Zmínky o médiích v poslaneckých projevech"
    )
).mark_bar(
    width=2
).encode(
    alt.X('datum:T', title=None),
    alt.Y('pomer:Q', axis=alt.Axis(domainOpacity=0, tickColor='#DCDDD6', tickCount=3, labelExpr="datum.label * 100 + ' %'", orient='right', title=None)),
    alt.Row('co:N', sort=poradi_medii, header=alt.Header(labelAngle=0, labelAlign='left', labelAnchor='start', labelOrient='top', labelFont='Asap'), title=None),
    alt.Color('max:N', legend=None, scale=alt.Scale(range=['#ECD410','#D55F50']))
).configure_axisX(
    grid=False, domain=False
).configure_axisY(
    grid=True, domain=False
).configure_view(
    stroke='transparent'
).resolve_axis(
    x="independent",
    y="independent"
).resolve_scale(
    y="shared"
)

graf_medii

alt.Chart(...)

In [253]:
me_to_neurazi(graf=graf_medii,kredity='vizualizace: iROZHLAS.cz | zdroj projevů: psp.cz', soubor='ruzna_media')

<figure>
    <a href="https://data.irozhlas.cz/grafy/ruzna_media.svg" target="_blank">
    <img src="https://data.irozhlas.cz/grafy/ruzna_media.svg" width="100%" alt="Graf s titulkem „Zmínky o médiích v poslaneckých projevech“. Další texty by měly být čitelné ze zdrojového souboru SVG." />
    </a>
    </figure>


## Konkrétní noviny

In [66]:
graf(pl.concat(
        [
            vyvoj_vyskytu(r"(?i) v noviná","v novinách"),
            vyvoj_vyskytu(r"Rudé\w{0,5} práv","Rudé právo"),
            vyvoj_vyskytu(r"MFD","MFD"),
            vyvoj_vyskytu(r"Národní politi[kc]","Národní politika"),
            vyvoj_vyskytu(r" v Přítomnosti","v Přítomnosti"),
            vyvoj_vyskytu(r"České\w{0,3} slov[ouě]","České slovo"),
            vyvoj_vyskytu(r"Svobodn\w{0,3} slov[ouě]","Svobodné slovo"),
            vyvoj_vyskytu(r"Pražsk\w{0,5} novin","Pražské noviny"),
            vyvoj_vyskytu(r"Lidov\w{0,5} novin","Lidové noviny"),
            vyvoj_vyskytu(r"Hospod\w{0,8} novin","Hospodářské noviny"),
            vyvoj_vyskytu(r"(?i)mlad\w{0,8} front\w{0,8} dnes","Mladá fronta DNES"),
            vyvoj_vyskytu(r"[\w,] Respekt","Respekt"),
            vyvoj_vyskytu(r"[\w,] Reflex","Reflex"),
            vyvoj_vyskytu(r"[\w,] Reflex","Reflex"),
            vyvoj_vyskytu(r" v Blesku","v Blesku"),
        ]
))

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)